In [1]:
# test_script.py
import threading
import socket
import time
from TmInstrument import TmInstrument
from TmInstrumentServer import TmInstrumentServer

def start_emulator(emulator):
    emulator.start()
    # Give the emulator some time to start
    time.sleep(2)

def test_commands():
    resource_name = 'TCPIP::127.0.0.1::5025::SOCKET'

    # try:
    vna = TmInstrument(resource_name)
    vna.timeout = 5000  # Timeout in milliseconds
    vna.opc_timeout = 5000  # OPC timeout in milliseconds

    # Send the command sequence and receive the responses
    commands = ["*IDN?", "CALC:DATA? SDATA"] #"CONF:VOLT:DC 10,0.001", "MEAS:VOLT:DC?", "SYST:ERR?"]
    responses = vna.send_command_sequence(commands,max_row_length=100)       

    time.sleep(2)

emulator = TmInstrumentServer()
emulator_thread = threading.Thread(target=start_emulator, args=(emulator,))
emulator_thread.start()

# Wait for the emulator to start
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.settimeout(5)  # Timeout in seconds
    while True:
        try:
            s.connect(("127.0.0.1", 5025))
            break
        except:
            pass

test_commands()

# Stop the emulator
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect(("127.0.0.1", 5025))
    s.sendall(b"DummyServer.stop")

emulator_thread.join()
time.sleep(2)

------------------------------------------Command Sequence------------------------------------------
Connected to:      Rohde&Schwarz,ZNB20,123456,1.00
> *IDN?
Rohde&Schwarz,ZNB20,123456,1.00
> CALC:DATA? SDATA
8.507416e-01,9.832794e-01,4.086000e-01,3.911884e-01,4.749336e-01,8.273829e-01,8.065995e-01,3.340855e
-01,1.541453e-01,8.401461e-01,5.342019e-01,4.836401e-01,6.469041e-01,8.818482e-01,7.370747e-01,7.450
974e-01,9.199255e-01,9.747917e-01,8.456256e-01,7.526771e-01,9.290774e-02,3.968811e-01,4.305573e-01,9
.426816e-01,1.536510e-01,3.073497e-01,5.995673e-01,9.006441e-01,4.254915e-01,4.935963e-01,6.560219e-
01,5.300489e-01,7.280158e-01,5.312966e-01,1.292859e-01,2.510128e-01,7.467577e-01,3.595232e-01,2. ...
Disconnected from: Rohde&Schwarz,ZNB20,123456,1.00
----------------------------------------2023-03-30 19:07:49-----------------------------------------
